# Protein - Structure Mapping, Alignments, and Visualization
This notebook gives an example of how to **map a single protein sequence to its structure**, along with conducting sequence alignments and visualizing the mutations.

<div class="alert alert-info">

**Input:** Protein ID + amino acid sequence + mutated sequence(s)

</div>

<div class="alert alert-info">

**Output:** Representative protein structure, sequence alignments, and visualization of mutations

</div>

## Imports

In [1]:
import sys
import logging

In [2]:
# Import the Protein class
from ssbio.core.protein import Protein

In [3]:
# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Logging

Set the logging level in `logger.setLevel(logging.<LEVEL_HERE>)` to specify how verbose you want the pipeline to be. Debug is most verbose.

- `CRITICAL`
     - Only really important messages shown
- `ERROR`
     - Major errors
- `WARNING`
     - Warnings that don't affect running of the pipeline
- `INFO` (default)
     - Info such as the number of structures mapped per gene
- `DEBUG`
     - Really detailed information that will print out a lot of stuff
     
<p><div class="alert alert-warning">**Warning:** `DEBUG` mode prints out a large amount of information, especially if you have a lot of genes. This may stall your notebook!</div></p>

In [4]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # SET YOUR LOGGING LEVEL HERE #

In [5]:
# Other logger stuff for Jupyter notebooks
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] %(levelname)s: %(message)s', datefmt="%Y-%m-%d %H:%M")
handler.setFormatter(formatter)
logger.handlers = [handler]

## Initialization of the project

Set these three things:

- `ROOT_DIR`
    - The directory where a folder named after your `PROTEIN_ID` will be created
- `PROTEIN_ID`
    - Your protein ID
- `PROTEIN_SEQ`
    - Your protein sequence
    
A directory will be created in `ROOT_DIR` with your `PROTEIN_ID` name. The folders are organized like so:
```
    ROOT_DIR
    └── PROTEIN_ID
        ├── sequences  # Protein sequence files, alignments, etc.
        └── structures  # Protein structure files, calculations, etc.

```

In [6]:
# SET FOLDERS AND DATA HERE
import tempfile
ROOT_DIR = tempfile.gettempdir()

PROTEIN_ID = 'SRR1753782_00918'
PROTEIN_SEQ = 'MSKQQIGVVGMAVMGRNLALNIESRGYTVSVFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRILLMVKAGAGTDAAIDSLKPYLEKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIMPGGQKDAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLSNEELANTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLITESVFARYISSLKAQRVAASKVLSGPKAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASDEYHWDLNYGEIAKIFRAGCIIRAQFLQKITDAYAENADIANLLLAPYFKKIADEYQQALRDVVAYAVQNGIPVPTFSAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRTDKEGIFHTEWLE'

In [7]:
# Create the Protein object
my_protein = Protein(ident=PROTEIN_ID, root_dir=ROOT_DIR, pdb_file_type='mmtf')

In [8]:
# Load the protein sequence
# This sets the loaded sequence as the representative one
my_protein.load_manual_sequence(seq=PROTEIN_SEQ, ident='WT', write_fasta_file=True, set_as_representative=True)

<SeqProp WT at 0x7ff5fa89fac8>

## Mapping sequence --> structure

Since the sequence has been provided, we just need to BLAST it to the PDB.

<p><div class="alert alert-info">**Note:** These methods do not download any 3D structure files.</div></p>

### Methods

In [9]:
# Mapping using BLAST
my_protein.blast_representative_sequence_to_pdb(seq_ident_cutoff=0.9, evalue=0.00001)
my_protein.df_pdb_blast.head()

['2zyd', '2zya', '3fwn', '2zyg']

,pdb_chain_id,hit_score,hit_evalue,hit_percent_similar,hit_percent_ident,hit_num_ident,hit_num_similar
pdb_id,,,,,,,
2zya,A,2319.0,0.0,0.987179,0.963675,451,462
2zya,B,2319.0,0.0,0.987179,0.963675,451,462
2zyd,A,2319.0,0.0,0.987179,0.963675,451,462
2zyd,B,2319.0,0.0,0.987179,0.963675,451,462
2zyg,A,2284.0,0.0,0.982906,0.950855,445,460


## Downloading and ranking structures

### Methods

<div class="alert alert-warning">

**Warning:** 
Downloading all PDBs takes a while, since they are also parsed for metadata. You can skip this step and just set representative structures below if you want to minimize the number of PDBs downloaded.

</div>

In [10]:
# Download all mapped PDBs and gather the metadata
my_protein.pdb_downloader_and_metadata()
my_protein.df_pdb_metadata.head(2)

['2zyd', '2zya', '3fwn', '2zyg']

,pdb_title,description,experimental_method,mapped_chains,resolution,chemicals,taxonomy_name,structure_file
pdb_id,,,,,,,,
2zya,Dimeric 6-phosphogluconate dehydrogenase compl...,"6-phosphogluconate dehydrogenase, decarboxylat...",X-RAY DIFFRACTION,A;B,1.6,6PG,Escherichia coli,2zya.mmtf
2zyd,Dimeric 6-phosphogluconate dehydrogenase compl...,"6-phosphogluconate dehydrogenase, decarboxylat...",X-RAY DIFFRACTION,A;B,1.5,GLO,Escherichia coli,2zyd.mmtf


In [11]:
# Set representative structures
my_protein.set_representative_structure()

<StructProp REP-2zyd at 0x7ff5f9ac6828>

## Loading and aligning new sequences

You can load additional sequences into this protein object and align them to the representative sequence.

### Methods

In [12]:
# Input your mutated sequence and load it
mutated_protein1_id = 'N17P_SNP'
mutated_protein1_seq = 'MSKQQIGVVGMAVMGRPLALNIESRGYTVSVFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRILLMVKAGAGTDAAIDSLKPYLEKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIMPGGQKDAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLSNEELANTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLITESVFARYISSLKAQRVAASKVLSGPKAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASDEYHWDLNYGEIAKIFRAGCIIRAQFLQKITDAYAENADIANLLLAPYFKKIADEYQQALRDVVAYAVQNGIPVPTFSAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRTDKEGIFHTEWLE'

my_protein.load_manual_sequence(ident=mutated_protein1_id, seq=mutated_protein1_seq)

<SeqProp N17P_SNP at 0x7ff5f90be2e8>

In [13]:
# Input another mutated sequence and load it
mutated_protein2_id = 'Q4S_N17P_SNP'
mutated_protein2_seq = 'MSKSQIGVVGMAVMGRPLALNIESRGYTVSVFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRILLMVKAGAGTDAAIDSLKPYLEKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIMPGGQKDAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLSNEELANTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLITESVFARYISSLKAQRVAASKVLSGPKAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASDEYHWDLNYGEIAKIFRAGCIIRAQFLQKITDAYAENADIANLLLAPYFKKIADEYQQALRDVVAYAVQNGIPVPTFSAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRTDKEGIFHTEWLE'

my_protein.load_manual_sequence(ident=mutated_protein2_id, seq=mutated_protein2_seq)

<SeqProp Q4S_N17P_SNP at 0x7ff5f8c9fb38>

In [14]:
# Conduct pairwise sequence alignments
my_protein.pairwise_align_sequences_to_representative()

In [15]:
# View IDs of all sequence alignments
[x.id for x in my_protein.sequence_alignments]

# View the stored information for one of the alignments
my_alignment = my_protein.sequence_alignments.get_by_id('SRR1753782_00918_N17P_SNP')
my_alignment.annotations
str(my_alignment[0].seq)
str(my_alignment[1].seq)

['WT_2zyd-A',
 'WT_2zyd-B',
 'SRR1753782_00918_N17P_SNP',
 'SRR1753782_00918_Q4S_N17P_SNP']

{'a_seq': 'WT',
 'b_seq': 'N17P_SNP',
 'deletions': [],
 'gaps': 0,
 'identity': 467,
 'insertions': [],
 'mutations': [('N', 17, 'P')],
 'percent_gaps': 0.0,
 'percent_identity': 99.8,
 'percent_similarity': 99.8,
 'score': 2381.0,
 'similarity': 467,
 'ssbio_type': 'seqalign'}

'MSKQQIGVVGMAVMGRNLALNIESRGYTVSVFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRILLMVKAGAGTDAAIDSLKPYLEKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIMPGGQKDAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLSNEELANTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLITESVFARYISSLKAQRVAASKVLSGPKAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASDEYHWDLNYGEIAKIFRAGCIIRAQFLQKITDAYAENADIANLLLAPYFKKIADEYQQALRDVVAYAVQNGIPVPTFSAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRTDKEGIFHTEWLE'

'MSKQQIGVVGMAVMGRPLALNIESRGYTVSVFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRILLMVKAGAGTDAAIDSLKPYLEKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIMPGGQKDAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLSNEELANTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLITESVFARYISSLKAQRVAASKVLSGPKAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASDEYHWDLNYGEIAKIFRAGCIIRAQFLQKITDAYAENADIANLLLAPYFKKIADEYQQALRDVVAYAVQNGIPVPTFSAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRTDKEGIFHTEWLE'

In [16]:
# Summarize all the mutations in all sequence alignments
s,f = my_protein.sequence_mutation_summary(alignment_type='seqalign')
print('Single mutations:')
s
print('---------------------')
print('Mutation fingerprints')
f

Single mutations:


{('N', 17, 'P'): ['N17P_SNP', 'Q4S_N17P_SNP'], ('Q', 4, 'S'): ['Q4S_N17P_SNP']}

---------------------
Mutation fingerprints


{(('N', 17, 'P'),): ['N17P_SNP'],
 (('Q', 4, 'S'), ('N', 17, 'P')): ['Q4S_N17P_SNP']}

## Some additional methods

### Getting binding site/other information from UniProt

In [17]:
import ssbio.databases.uniprot

In [18]:
this_examples_uniprot = 'P14062'
sites = ssbio.databases.uniprot.uniprot_sites(this_examples_uniprot)
my_protein.representative_sequence.features = sites
my_protein.representative_sequence.features

[SeqFeature(FeatureLocation(ExactPosition(9), ExactPosition(15)), type='Nucleotide binding'),
 SeqFeature(FeatureLocation(ExactPosition(32), ExactPosition(35)), type='Nucleotide binding'),
 SeqFeature(FeatureLocation(ExactPosition(73), ExactPosition(76)), type='Nucleotide binding'),
 SeqFeature(FeatureLocation(ExactPosition(127), ExactPosition(130)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(185), ExactPosition(187)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(182), ExactPosition(183)), type='Active site'),
 SeqFeature(FeatureLocation(ExactPosition(189), ExactPosition(190)), type='Active site'),
 SeqFeature(FeatureLocation(ExactPosition(101), ExactPosition(102)), type='Binding site'),
 SeqFeature(FeatureLocation(ExactPosition(101), ExactPosition(102)), type='Binding site'),
 SeqFeature(FeatureLocation(ExactPosition(190), ExactPosition(191)), type='Binding site'),
 SeqFeature(FeatureLocation(ExactPosition(259), ExactPosition(260)), type='Binding site'),
 S

## Mapping sequence residue numbers to structure residue numbers

### Methods

In [19]:
# Returns a dictionary mapping sequence residue numbers to structure residue identifiers
# Will warn you if residues are not present in the structure
structure_sites = my_protein.map_seqprop_resnums_to_structprop_resnums(resnums=[1,3,45], 
                                                                       use_representatives=True)
structure_sites

[2017-11-21 19:21] [ssbio.core.protein] WARNING: REP-2zyd-A, 1: structure file does not contain coordinates for this residue


{3: 3, 45: 45}

## Viewing structures

The awesome package [nglview](https://github.com/arose/nglview) is utilized as a backend for viewing structures within a Jupyter notebook. `ssbio` view functions will either return a `NGLWidget` object, which is the same as using `nglview` like the below example, or act upon the widget object itself.

```python
# This is how NGLview usually works - it will load a structure file and return a NGLWidget "view" object.
import nglview
view = nglview.show_structure_file(my_protein.representative_structure.structure_path)
view
```

### Methods

In [20]:
# View just the structure
view = my_protein.representative_structure.view_structure()
view

NGLWidget()

In [21]:
# Map the mutations on the visualization (scale increased) - will show up on the above view
my_protein.add_mutations_to_nglview(view=view, alignment_type='seqalign', scale_range=(4,7), 
                                    use_representatives=True)

[2017-11-21 19:21] [ssbio.protein.structure.structprop] INFO: Selection: ( :A ) and not hydrogen and 17
[2017-11-21 19:21] [ssbio.protein.structure.structprop] INFO: Selection: ( :A ) and not hydrogen and 4


In [22]:
# Add sites as shown above in the table to the view
my_protein.add_features_to_nglview(view=view, use_representatives=True)

[2017-11-21 19:21] [ssbio.core.protein] INFO: Active site at sequence residue 183, structure residue 183
[2017-11-21 19:21] [ssbio.core.protein] INFO: Active site at sequence residue 190, structure residue 190
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 102, structure residue 102
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 102, structure residue 102
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 191, structure residue 191
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 260, structure residue 260
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 287, structure residue 287
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 445, structure residue 445
[2017-11-21 19:21] [ssbio.core.protein] INFO: Binding site at sequence residue 451, structure residue 451


## Saving

In [23]:
import os.path as op
my_protein.save_json(op.join(my_protein.protein_dir, '{}.json'.format(my_protein.id)))

[2017-11-21 19:21] [root] WARNING: json-tricks: numpy scalar serialization is experimental and may work differently in future versions
[2017-11-21 19:21] [ssbio.core.io] INFO: Saved <class 'ssbio.core.protein.Protein'> (id: SRR1753782_00918) to /tmp/SRR1753782_00918/SRR1753782_00918.json
